In [2]:
!pip install -q transformers jiwer accelerate soundfile librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.7 MB/s eta 0:00:00a 0:00:01


In [3]:
"""
Speech-to-Text Model Comparison Script
Evaluates multiple STT models on accuracy, speed, and resource usage
Uses LibriSpeech dataset from Kaggle

Required installations:
!pip install transformers jiwer accelerate soundfile librosa
"""

import time
import torch
import numpy as np
import jiwer
from transformers import (
    AutoModelForSpeechSeq2Seq, 
    AutoProcessor,
    pipeline
)
import gc
import pandas as pd
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

# Ensure audio libraries are available
try:
    import soundfile as sf
    import librosa
except ImportError:
    print("Installing required audio libraries...")
    import subprocess
    subprocess.check_call(['pip', 'install', '-q', 'soundfile', 'librosa'])
    import soundfile as sf
    import librosa

# Configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TORCH_DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

class STTModelEvaluator:
    """Evaluate Speech-to-Text models"""
    
    def __init__(self):
        self.results = []
        self.device = DEVICE
        self.torch_dtype = TORCH_DTYPE
        
    def load_librispeech_dataset(self, base_path, max_samples=None):
        """Load LibriSpeech dataset from folder structure
        
        Args:
            base_path: Path to LibriSpeech folder (e.g., '/kaggle/input/benchmark/LibriSpeech/test-clean')
            max_samples: Maximum number of samples to load (None = load all)
        
        Expected structure:
            base_path/
                speaker_id/
                    chapter_id/
                        speaker_id-chapter_id-utterance_id.flac
                        speaker_id-chapter_id.trans.txt
        """
        print(f"Loading LibriSpeech dataset from: {base_path}")
        
        import os
        from pathlib import Path
        
        samples = []
        transcripts = {}
        
        # Walk through the directory structure
        base_path = Path(base_path)
        
        # Find all .trans.txt files
        trans_files = list(base_path.rglob("*.trans.txt"))
        print(f"Found {len(trans_files)} transcript files")
        
        # Parse transcript files
        for trans_file in trans_files:
            with open(trans_file, 'r', encoding='utf-8') as f:
                for line in f:
                    parts = line.strip().split(' ', 1)
                    if len(parts) == 2:
                        utterance_id, text = parts
                        transcripts[utterance_id] = text.strip()
        
        print(f"Loaded {len(transcripts)} transcripts")
        
        # Find all .flac audio files
        audio_files = list(base_path.rglob("*.flac"))
        print(f"Found {len(audio_files)} audio files")
        
        # Load audio files with their transcripts
        loaded_count = 0
        for audio_file in audio_files:
            # Extract utterance ID from filename
            utterance_id = audio_file.stem  # e.g., "1089-134691-0000"
            
            if utterance_id in transcripts:
                try:
                    # Load audio
                    audio, sr = librosa.load(str(audio_file), sr=16000)
                    
                    samples.append({
                        'audio': audio,
                        'sampling_rate': sr,
                        'reference': transcripts[utterance_id].upper(),
                        'file': str(audio_file.name)
                    })
                    
                    loaded_count += 1
                    print(f"  Loaded {loaded_count} samples...", end='\r')
                    
                    # Check if we've reached max_samples
                    if max_samples and loaded_count >= max_samples:
                        break
                        
                except Exception as e:
                    print(f"\n  Error loading {audio_file}: {e}")
                    continue
        
        print(f"\n✓ Successfully loaded {len(samples)} audio samples")
        
        if len(samples) == 0:
            print("❌ No samples loaded. Please check the dataset path.")
            print(f"   Looking in: {base_path}")
            print(f"   Expected structure: speaker_id/chapter_id/*.flac")
        
        return samples
    
    def load_multiple_speakers(self, base_path, speaker_folders, max_samples_per_speaker=None):
        """Load samples from multiple speaker folders
        
        Args:
            base_path: Base path to LibriSpeech (e.g., '/kaggle/input/benchmark/LibriSpeech/test-clean')
            speaker_folders: List of speaker/chapter paths (e.g., ['1089/134691', '1089/134686'])
            max_samples_per_speaker: Max samples per speaker folder
        """
        print(f"Loading samples from {len(speaker_folders)} speaker folders...")
        
        all_samples = []
        
        for speaker_folder in speaker_folders:
            folder_path = f"{base_path}/{speaker_folder}"
            print(f"\nLoading from: {folder_path}")
            
            samples = self.load_librispeech_dataset(folder_path, max_samples_per_speaker)
            all_samples.extend(samples)
        
        print(f"\n✓ Total samples loaded: {len(all_samples)}")
        return all_samples
    
    def evaluate_whisper_large_v3(self, test_data: List[Dict]) -> Dict:
        """Evaluate OpenAI Whisper Large v3"""
        print("\n" + "="*60)
        print("Evaluating: Whisper Large v3")
        print("="*60)
        
        model_id = "openai/whisper-large-v3"
        
        try:
            # Load model
            start_load = time.time()
            model = AutoModelForSpeechSeq2Seq.from_pretrained(
                model_id,
                torch_dtype=self.torch_dtype,
                low_cpu_mem_usage=True,
                use_safetensors=True
            )
            model.to(self.device)
            
            processor = AutoProcessor.from_pretrained(model_id)
            
            pipe = pipeline(
                "automatic-speech-recognition",
                model=model,
                tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,
                max_new_tokens=128,
                chunk_length_s=30,
                batch_size=16,
                return_timestamps=False,
                torch_dtype=self.torch_dtype,
                device=self.device,
            )
            load_time = time.time() - start_load
            print(f"✓ Model loaded in {load_time:.2f}s")
            
            # Run inference
            predictions = []
            references = []
            inference_times = []
            
            for idx, sample in enumerate(test_data):
                start = time.time()
                result = pipe(sample['audio'])
                inference_times.append(time.time() - start)
                
                pred_text = result['text'].strip().upper()
                predictions.append(pred_text)
                references.append(sample['reference'].upper())
                
                print(f"  Sample {idx+1}/{len(test_data)}: {inference_times[-1]:.3f}s")
                if idx < 3:  # Show first 3 predictions
                    print(f"    Predicted: {pred_text[:60]}...")
                    print(f"    Reference: {sample['reference'][:60]}...")
            
            # Calculate metrics
            wer = jiwer.wer(references, predictions)
            avg_inference_time = np.mean(inference_times)
            
            print(f"\n✓ WER: {wer*100:.2f}%")
            print(f"✓ Avg inference: {avg_inference_time:.3f}s")
            
            results = {
                'model': 'Whisper Large v3',
                'wer': wer * 100,
                'load_time': load_time,
                'avg_inference_time': avg_inference_time,
                'total_inference_time': sum(inference_times),
                'model_size': 'Large (~1.5GB)',
                'notes': 'Multilingual, 99+ languages'
            }
            
            # Cleanup
            del model, processor, pipe
            gc.collect()
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            
            return results
            
        except Exception as e:
            print(f"❌ Error evaluating Whisper Large v3: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def evaluate_distil_whisper(self, test_data: List[Dict]) -> Dict:
        """Evaluate Distil-Whisper Large v3"""
        print("\n" + "="*60)
        print("Evaluating: Distil-Whisper Large v3")
        print("="*60)
        
        model_id = "distil-whisper/distil-large-v3"
        
        try:
            start_load = time.time()
            model = AutoModelForSpeechSeq2Seq.from_pretrained(
                model_id,
                torch_dtype=self.torch_dtype,
                low_cpu_mem_usage=True,
                use_safetensors=True
            )
            model.to(self.device)
            
            processor = AutoProcessor.from_pretrained(model_id)
            
            pipe = pipeline(
                "automatic-speech-recognition",
                model=model,
                tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,
                max_new_tokens=128,
                chunk_length_s=30,
                batch_size=16,
                torch_dtype=self.torch_dtype,
                device=self.device,
            )
            load_time = time.time() - start_load
            print(f"✓ Model loaded in {load_time:.2f}s")
            
            predictions = []
            references = []
            inference_times = []
            
            for idx, sample in enumerate(test_data):
                start = time.time()
                result = pipe(sample['audio'])
                inference_times.append(time.time() - start)
                
                pred_text = result['text'].strip().upper()
                predictions.append(pred_text)
                references.append(sample['reference'].upper())
                
                print(f"  Sample {idx+1}/{len(test_data)}: {inference_times[-1]:.3f}s")
                if idx < 3:
                    print(f"    Predicted: {pred_text[:60]}...")
                    print(f"    Reference: {sample['reference'][:60]}...")
            
            wer = jiwer.wer(references, predictions)
            avg_inference_time = np.mean(inference_times)
            
            print(f"\n✓ WER: {wer*100:.2f}%")
            print(f"✓ Avg inference: {avg_inference_time:.3f}s")
            
            results = {
                'model': 'Distil-Whisper Large v3',
                'wer': wer * 100,
                'load_time': load_time,
                'avg_inference_time': avg_inference_time,
                'total_inference_time': sum(inference_times),
                'model_size': 'Medium (~756MB)',
                'notes': '6x faster than Whisper, minimal accuracy loss'
            }
            
            del model, processor, pipe
            gc.collect()
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            
            return results
            
        except Exception as e:
            print(f"❌ Error evaluating Distil-Whisper: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def evaluate_whisper_base(self, test_data: List[Dict]) -> Dict:
        """Evaluate Whisper Base (faster, smaller)"""
        print("\n" + "="*60)
        print("Evaluating: Whisper Base")
        print("="*60)
        
        model_id = "openai/whisper-base"
        
        try:
            start_load = time.time()
            model = AutoModelForSpeechSeq2Seq.from_pretrained(
                model_id,
                torch_dtype=self.torch_dtype,
                low_cpu_mem_usage=True,
                use_safetensors=True
            )
            model.to(self.device)
            
            processor = AutoProcessor.from_pretrained(model_id)
            
            pipe = pipeline(
                "automatic-speech-recognition",
                model=model,
                tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,
                max_new_tokens=128,
                chunk_length_s=30,
                batch_size=16,
                torch_dtype=self.torch_dtype,
                device=self.device,
            )
            load_time = time.time() - start_load
            print(f"✓ Model loaded in {load_time:.2f}s")
            
            predictions = []
            references = []
            inference_times = []
            
            for idx, sample in enumerate(test_data):
                start = time.time()
                result = pipe(sample['audio'])
                inference_times.append(time.time() - start)
                
                pred_text = result['text'].strip().upper()
                predictions.append(pred_text)
                references.append(sample['reference'].upper())
                
                print(f"  Sample {idx+1}/{len(test_data)}: {inference_times[-1]:.3f}s")
                if idx < 3:
                    print(f"    Predicted: {pred_text[:60]}...")
                    print(f"    Reference: {sample['reference'][:60]}...")
            
            wer = jiwer.wer(references, predictions)
            avg_inference_time = np.mean(inference_times)
            
            print(f"\n✓ WER: {wer*100:.2f}%")
            print(f"✓ Avg inference: {avg_inference_time:.3f}s")
            
            results = {
                'model': 'Whisper Base',
                'wer': wer * 100,
                'load_time': load_time,
                'avg_inference_time': avg_inference_time,
                'total_inference_time': sum(inference_times),
                'model_size': 'Small (~140MB)',
                'notes': 'Fast inference, good for real-time'
            }
            
            del model, processor, pipe
            gc.collect()
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            
            return results
            
        except Exception as e:
            print(f"❌ Error evaluating Whisper Base: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def run_comparison(self, test_data, num_samples=10):
        """Run complete comparison"""
        print("="*80)
        print("SPEECH-TO-TEXT MODEL COMPARISON")
        print("="*80)
        print(f"Device: {self.device}")
        print(f"Torch dtype: {self.torch_dtype}")
        
        if len(test_data) == 0:
            print("❌ No test data available. Please provide audio samples.")
            return None
        
        print(f"\nRunning tests on {len(test_data)} samples...")
        
        # Evaluate models
        results = []
        
        # Whisper Base (fastest)
        result = self.evaluate_whisper_base(test_data)
        if result:
            results.append(result)
        
        # Distil-Whisper (balanced)
        result = self.evaluate_distil_whisper(test_data)
        if result:
            results.append(result)
        
        # Whisper Large v3 (most accurate)
        result = self.evaluate_whisper_large_v3(test_data)
        if result:
            results.append(result)
        
        # Create results dataframe
        if results:
            df = pd.DataFrame(results)
            return df
        else:
            print("❌ No models completed successfully")
            return None
    
    def display_results(self, df: pd.DataFrame):
        """Display comparison results"""
        if df is None or len(df) == 0:
            print("No results to display")
            return
            
        print("\n" + "="*80)
        print("RESULTS SUMMARY")
        print("="*80)
        print(f"\nMetrics:")
        print(df.to_string(index=False))
        
        print("\n" + "-"*80)
        print("KEY METRICS:")
        print("-"*80)
        print("WER (Word Error Rate): Lower is better (% of words incorrectly transcribed)")
        print("Avg Inference Time: Time per audio sample (seconds)")
        print("Load Time: Model loading time (seconds)")
        
        print("\n" + "-"*80)
        print("RECOMMENDATIONS:")
        print("-"*80)
        
        # Find best models
        best_accuracy = df.loc[df['wer'].idxmin()]
        best_speed = df.loc[df['avg_inference_time'].idxmin()]
        
        print(f"\n🎯 Best Accuracy: {best_accuracy['model']} (WER: {best_accuracy['wer']:.2f}%)")
        print(f"⚡ Fastest: {best_speed['model']} ({best_speed['avg_inference_time']:.3f}s per sample)")
        
        print("\n💡 Use Case Recommendations:")
        print("   • Real-time applications → Whisper Base")
        print("   • Balanced performance → Distil-Whisper Large v3")
        print("   • Maximum accuracy → Whisper Large v3")
        print("   • Production/API → Deepgram Nova-2 (requires API key)")
        
        # Save results
        df.to_csv('stt_comparison_results.csv', index=False)
        print("\n✅ Results saved to: stt_comparison_results.csv")


# Main execution
if __name__ == "__main__":
    print("""
    ╔════════════════════════════════════════════════════════════════╗
    ║         SPEECH-TO-TEXT MODEL COMPARISON TOOL                   ║
    ║                                                                ║
    ║  This script compares multiple STT models on:                  ║
    ║  • Accuracy (Word Error Rate)                                  ║
    ║  • Speed (Inference Time)                                      ║
    ║  • Model Size                                                  ║
    ╚════════════════════════════════════════════════════════════════╝
    """)
    
    evaluator = STTModelEvaluator()
    
    # Load LibriSpeech dataset from multiple speaker folders
    test_data = evaluator.load_multiple_speakers(
        base_path='/kaggle/input/benchmark/LibriSpeech/test-clean',
        speaker_folders=['1089/134691', '1089/134686'],
        max_samples_per_speaker=10  # 10 samples from each folder = 20 total
    )
    
    # Run comparison
    results_df = evaluator.run_comparison(test_data)
    
    # Display results
    evaluator.display_results(results_df)
    
    print("\n" + "="*80)
    print("✅ Comparison complete!")
    print("="*80)

2026-01-23 06:59:56.582896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769151596.775146      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769151596.833781      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769151597.395993      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769151597.396040      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769151597.396043      55 computation_placer.cc:177] computation placer alr


    ╔════════════════════════════════════════════════════════════════╗
    ║         SPEECH-TO-TEXT MODEL COMPARISON TOOL                   ║
    ║                                                                ║
    ║  This script compares multiple STT models on:                  ║
    ║  • Accuracy (Word Error Rate)                                  ║
    ║  • Speed (Inference Time)                                      ║
    ║  • Model Size                                                  ║
    ╚════════════════════════════════════════════════════════════════╝
    
Loading samples from 2 speaker folders...

Loading from: /kaggle/input/benchmark/LibriSpeech/test-clean/1089/134691
Loading LibriSpeech dataset from: /kaggle/input/benchmark/LibriSpeech/test-clean/1089/134691
Found 1 transcript files
Loaded 26 transcripts
Found 26 audio files
  Loaded 10 samples...
✓ Successfully loaded 10 audio samples

Loading from: /kaggle/input/benchmark/LibriSpeech/test-clean/1089/134686
Loading Libri

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


✓ Model loaded in 16.68s
  Sample 1/20: 4.614s
    Predicted: IDLE AND EMBITTERING FINALLY TO ARGUE AGAINST HIS OWN DISPAS...
    Reference: IDLE AND EMBITTERING FINALLY TO ARGUE AGAINST HIS OWN DISPAS...
  Sample 2/20: 2.421s
    Predicted: THE PRIDE OF THAT DIM IMAGE BROUGHT BACK TO HIS MIND THE DIG...
    Reference: THE PRIDE OF THAT DIM IMAGE BROUGHT BACK TO HIS MIND THE DIG...
  Sample 3/20: 2.607s
    Predicted: FOR A FULL HOUR HE HAD PACED UP AND DOWN, WAITING, BUT HE CO...
    Reference: FOR A FULL HOUR HE HAD PACED UP AND DOWN WAITING BUT HE COUL...
  Sample 4/20: 2.354s
  Sample 5/20: 1.843s
  Sample 6/20: 2.157s
  Sample 7/20: 1.715s
  Sample 8/20: 4.016s
  Sample 9/20: 1.782s
  Sample 10/20: 1.397s
  Sample 11/20: 2.287s
  Sample 12/20: 2.988s
  Sample 13/20: 2.744s
  Sample 14/20: 2.290s
  Sample 15/20: 2.416s
  Sample 16/20: 2.224s
  Sample 17/20: 2.416s
  Sample 18/20: 3.373s
  Sample 19/20: 2.417s
  Sample 20/20: 1.592s

✓ WER: 12.58%
✓ Avg inference: 2.483s

RESULTS SU

In [15]:
"""
Fixed STT benchmark: uses nvidia/canary-1b-flash for Canary (has HF preprocessor)
Requires: bitsandbytes, accelerate, transformers, jiwer, librosa, soundfile
"""

import time
import gc
import torch
import numpy as np
import pandas as pd
import jiwer
import librosa
from typing import List, Dict

from transformers import (
    pipeline,
    BitsAndBytesConfig,
    AutoModelForSpeechSeq2Seq,
    AutoProcessor,
)

# -------------------------
# Config
# -------------------------
USE_CUDA = torch.cuda.is_available()
DEVICE_IDX = 0 if USE_CUDA else -1  # pipeline expects GPU index or -1 for CPU
DTYPE = torch.float16 if USE_CUDA else torch.float32

# -------------------------
# Evaluator
# -------------------------
class STTModelEvaluator:
    def __init__(self):
        self.device_idx = DEVICE_IDX
        self.dtype = DTYPE

    # -------------------------
    # Data loading
    # -------------------------
    def load_librispeech_dataset(self, base_path, max_samples=None):
        from pathlib import Path
        samples = []
        transcripts = {}
        base_path = Path(base_path)

        for trans_file in base_path.rglob("*.trans.txt"):
            with open(trans_file, "r", encoding="utf-8") as f:
                for line in f:
                    uid, text = line.strip().split(" ", 1)
                    transcripts[uid] = text.strip()

        count = 0
        for audio_file in base_path.rglob("*.flac"):
            uid = audio_file.stem
            if uid in transcripts:
                audio, sr = librosa.load(str(audio_file), sr=16000)
                samples.append({"audio": audio, "reference": transcripts[uid].upper()})
                count += 1
                if max_samples and count >= max_samples:
                    break

        print(f"Loaded {len(samples)} samples from {base_path}")
        return samples

    def load_multiple_speakers(self, base_path, speaker_folders, max_samples_per_speaker=3):
        all_samples = []
        for folder in speaker_folders:
            all_samples.extend(
                self.load_librispeech_dataset(f"{base_path}/{folder}", max_samples=max_samples_per_speaker)
            )
        print(f"Total samples: {len(all_samples)}")
        return all_samples

    # -------------------------
    # String-based pipeline evaluation (for models that work with simple pipeline)
    # -------------------------
    def evaluate_with_pipeline_string(self, model_id: str, name: str, data: List[Dict], size: str, notes: str):
        print(f"\nEvaluating: {name} (pipeline string)")
        start_load = time.time()

        pipe = pipeline(
            "automatic-speech-recognition",
            model=model_id,
            device=self.device_idx,
            torch_dtype=self.dtype,
        )

        load_time = time.time() - start_load
        preds, refs, times = [], [], []

        for i, sample in enumerate(data):
            t0 = time.time()
            out = pipe(sample["audio"])
            times.append(time.time() - t0)
            preds.append(out.get("text", "").strip().upper())
            refs.append(sample["reference"].upper())
            print(f"  Sample {i+1}/{len(data)}: {times[-1]:.3f}s")

        wer = jiwer.wer(refs, preds)
        del pipe
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return {
            "model": name,
            "wer": wer * 100,
            "load_time": load_time,
            "avg_inference_time": float(np.mean(times)),
            "total_inference_time": float(sum(times)),
            "model_size": size,
            "notes": notes,
        }

    # -------------------------
    # Manual load for quantized models (e.g., Canary)
    # -------------------------
    def evaluate_with_manual_load(self, model_id: str, name: str, data: List[Dict], size: str, notes: str, quantize_8bit: bool = False):
        """
        Load processor + model via from_pretrained; for 8-bit use BitsAndBytesConfig + device_map='auto'.
        If processor load fails for the given model_id, attempt fallback to 'nvidia/canary-1b-flash'.
        """
        print(f"\nEvaluating: {name} (manual load, quantize_8bit={quantize_8bit})")

        # Try to load processor; if it fails, attempt fallback to canary-1b-flash
        try:
            processor = AutoProcessor.from_pretrained(model_id)
        except Exception as e:
            print(f"Warning: failed to load processor for {model_id}: {e}")
            fallback = "nvidia/canary-1b-flash"
            if model_id != fallback:
                print(f"Attempting fallback processor: {fallback}")
                try:
                    processor = AutoProcessor.from_pretrained(fallback)
                    model_id = fallback
                    print(f"Using fallback model_id: {model_id}")
                except Exception as e2:
                    raise RuntimeError(f"Failed to load processor for both {model_id} and fallback {fallback}: {e2}")
            else:
                raise RuntimeError(f"Failed to load processor for {model_id}: {e}")

        # Prepare quantization config if requested
        quant_config = BitsAndBytesConfig(load_in_8bit=True) if quantize_8bit else None

        start_load = time.time()
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            model_id,
            quantization_config=quant_config,
            device_map="auto" if quantize_8bit and torch.cuda.is_available() else None,
            torch_dtype=self.dtype,
            low_cpu_mem_usage=True,
        )

        pipe = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            device=self.device_idx,
            torch_dtype=self.dtype,
        )

        load_time = time.time() - start_load

        preds, refs, times = [], [], []
        for i, sample in enumerate(data):
            t0 = time.time()
            out = pipe(sample["audio"])
            times.append(time.time() - t0)
            preds.append(out.get("text", "").strip().upper())
            refs.append(sample["reference"].upper())
            print(f"  Sample {i+1}/{len(data)}: {times[-1]:.3f}s")

        wer = jiwer.wer(refs, preds)

        # cleanup
        del pipe, model, processor
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return {
            "model": name,
            "wer": wer * 100,
            "load_time": load_time,
            "avg_inference_time": float(np.mean(times)),
            "total_inference_time": float(sum(times)),
            "model_size": size,
            "notes": notes,
        }

    # -------------------------
    # Run full suite
    # -------------------------
    def run_all(self, data: List[Dict]):
        results = []

        results.append(self.evaluate_with_pipeline_string(
            "openai/whisper-base", "Whisper Base", data, "~140MB", "Fastest baseline"
        ))

        results.append(self.evaluate_with_pipeline_string(
            "distil-whisper/distil-large-v3", "Distil-Whisper Large v3", data, "~756MB", "Speed/accuracy tradeoff"
        ))

        results.append(self.evaluate_with_pipeline_string(
            "openai/whisper-large-v3", "Whisper Large v3", data, "~1.5GB", "High accuracy, slower"
        ))

        results.append(self.evaluate_with_pipeline_string(
            "facebook/wav2vec2-large-960h-lv60-self", "wav2vec2 Large", data, "~1.2GB", "CTC model; may need fine-tuning"
        ))

        # Use Canary Flash variant (has preprocessor_config.json on HF)
        results.append(self.evaluate_with_manual_load(
            "nvidia/canary-1b-flash", "Canary 1B Flash", data, "~1.2GB", "NVIDIA Canary (8-bit load)", quantize_8bit=True
        ))

        return pd.DataFrame(results)


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    evaluator = STTModelEvaluator()

    test_data = evaluator.load_multiple_speakers(
        base_path="/kaggle/input/benchmark/LibriSpeech/test-clean",
        speaker_folders=["1089/134691", "1089/134686"],
        max_samples_per_speaker=3
    )

    df = evaluator.run_all(test_data)
    print("\n====== RESULTS ======")
    print(df.to_string(index=False))
    df.to_csv("stt_results_canary_fixed.csv", index=False)
    print("\nSaved: stt_results_canary_fixed.csv")


Loaded 3 samples from /kaggle/input/benchmark/LibriSpeech/test-clean/1089/134691
Loaded 3 samples from /kaggle/input/benchmark/LibriSpeech/test-clean/1089/134686
Total samples: 6

Evaluating: Whisper Base (pipeline string)


Device set to use cuda:0


  Sample 1/6: 0.731s
  Sample 2/6: 0.295s
  Sample 3/6: 0.299s
  Sample 4/6: 0.268s
  Sample 5/6: 0.395s
  Sample 6/6: 0.307s

Evaluating: Distil-Whisper Large v3 (pipeline string)


Device set to use cuda:0


  Sample 1/6: 0.845s
  Sample 2/6: 0.580s
  Sample 3/6: 0.595s
  Sample 4/6: 0.556s
  Sample 5/6: 0.653s
  Sample 6/6: 0.588s

Evaluating: Whisper Large v3 (pipeline string)


Device set to use cuda:0


  Sample 1/6: 4.625s
  Sample 2/6: 2.426s
  Sample 3/6: 2.618s
  Sample 4/6: 2.295s
  Sample 5/6: 3.007s
  Sample 6/6: 2.744s

Evaluating: wav2vec2 Large (pipeline string)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


  Sample 1/6: 0.127s
  Sample 2/6: 0.052s
  Sample 3/6: 0.050s
  Sample 4/6: 0.050s
  Sample 5/6: 0.090s
  Sample 6/6: 0.052s

Evaluating: Canary 1B Flash (manual load, quantize_8bit=True)

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`


RuntimeError: Failed to load processor for nvidia/canary-1b-flash: The checkpoint you are trying to load has model type `fastconformer` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [18]:
!pip install -U nemo_toolkit[asr] soundfile librosa jiwer accelerate bitsandbytes transformers
!apt-get update -y && apt-get install -y libsndfile1 ffmpeg
!pip install sentence-transformers


  Using cached jiwer-4.0.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease     
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease                     
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lis

In [25]:
import time
import gc
import torch
import numpy as np
import pandas as pd
import jiwer
import librosa
import soundfile as sf
import tempfile
import random
import os
import contextlib
import warnings

# ==========================
# SILENCE ALL WARNINGS / LOGS
# ==========================
warnings.filterwarnings("ignore")
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import pipeline
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()
hf_logging.disable_progress_bar()

import nemo.collections.asr as nemo_asr
from sentence_transformers import SentenceTransformer, util


# ==========================
# CONFIG
# ==========================
DEVICE = 0 if torch.cuda.is_available() else -1
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32
SEM_MODEL = SentenceTransformer("all-MiniLM-L6-v2")


# ==========================
# UTILS
# ==========================
def add_noise(audio, noise_level=0.005):
    noise = np.random.randn(len(audio))
    return audio + noise_level * noise


def semantic_similarity(refs, preds):
    ref_emb = SEM_MODEL.encode(refs, convert_to_tensor=True)
    pred_emb = SEM_MODEL.encode(preds, convert_to_tensor=True)
    sims = util.cos_sim(ref_emb, pred_emb).diagonal()
    return float(sims.mean())


# ==========================
# BENCHMARK CLASS
# ==========================
class STTBenchmark:

    def load_librispeech_dataset(self, base_path, max_samples=1000):
        from pathlib import Path

        base_path = Path(base_path)
        transcripts = {}
        audio_pairs = []

        for trans_file in base_path.rglob("*.trans.txt"):
            with open(trans_file) as f:
                for line in f:
                    uid, text = line.strip().split(" ", 1)
                    transcripts[uid] = text.strip()

        for audio_file in base_path.rglob("*.flac"):
            uid = audio_file.stem
            if uid in transcripts:
                audio_pairs.append((audio_file, transcripts[uid]))

        random.shuffle(audio_pairs)
        audio_pairs = audio_pairs[:max_samples]

        samples = []
        for audio_file, text in audio_pairs:
            audio, sr = librosa.load(str(audio_file), sr=16000)
            samples.append({
                "audio": audio,
                "reference": text.upper(),
                "duration": len(audio) / sr
            })

        print(f"Loaded {len(samples)} random samples")
        return samples


    # --------------------------
    # TRANSFORMERS MODELS
    # --------------------------
    def eval_transformers(self, model_id, name, data, size, notes):
        print(f"\nEvaluating: {name}")

        start = time.time()
        pipe = pipeline(
            "automatic-speech-recognition",
            model=model_id,
            device=DEVICE,
            torch_dtype=DTYPE
        )
        load_time = time.time() - start

        preds, refs, times, rtfs = [], [], [], []

        for s in data:
            t0 = time.time()

            # silence stderr completely during inference
            with open(os.devnull, "w") as fnull:
                with contextlib.redirect_stderr(fnull):
                    out = pipe(
                        s["audio"],
                        chunk_length_s=30,
                        stride_length_s=5
                    )

            t = time.time() - t0

            pred = out["text"].strip().upper()
            preds.append(pred)
            refs.append(s["reference"])
            times.append(t)
            rtfs.append(t / s["duration"])

        fname = f"predictions_{name.lower().replace(' ', '_')}.csv"
        pd.DataFrame({"reference": refs, "prediction": preds}).to_csv(fname, index=False)
        print(f"Saved predictions → {fname}")

        wer = jiwer.wer(refs, preds)
        sem = semantic_similarity(refs, preds)

        noisy_preds = []
        for s in data:
            with open(os.devnull, "w") as fnull:
                with contextlib.redirect_stderr(fnull):
                    out = pipe(add_noise(s["audio"]), chunk_length_s=30, stride_length_s=5)
            noisy_preds.append(out["text"].strip().upper())

        robust = jiwer.wer(refs, noisy_preds)

        del pipe
        gc.collect()
        torch.cuda.empty_cache()

        return {
            "model": name,
            "wer": wer * 100,
            "semantic_similarity": sem,
            "avg_rtf": float(np.mean(rtfs)),
            "robust_wer": robust * 100,
            "load_time": load_time,
            "avg_inference_time": float(np.mean(times)),
            "total_inference_time": float(sum(times)),
            "model_size": size,
            "notes": notes
        }


    # --------------------------
    # CANARY (NEMO)
    # --------------------------
    def eval_canary_nemo(self, data):
        print("\nEvaluating: Canary 1B Flash (NeMo)")

        start = time.time()
        model = nemo_asr.models.ASRModel.from_pretrained("nvidia/canary-1b-flash")
        if torch.cuda.is_available():
            model = model.to("cuda")
        load_time = time.time() - start

        preds, refs, times, rtfs = [], [], [], []

        for s in data:
            with tempfile.NamedTemporaryFile(suffix=".wav") as tmp:
                sf.write(tmp.name, s["audio"], 16000)

                t0 = time.time()
                out = model.transcribe([tmp.name], source_lang="en", target_lang="en")
                t = time.time() - t0

                text = out[0] if isinstance(out[0], str) else out[0].text
                preds.append(text.strip().upper())
                refs.append(s["reference"])
                times.append(t)
                rtfs.append(t / s["duration"])

        pd.DataFrame({"reference": refs, "prediction": preds}).to_csv(
            "predictions_canary_1b_flash.csv", index=False
        )
        print("Saved predictions → predictions_canary_1b_flash.csv")

        wer = jiwer.wer(refs, preds)
        sem = semantic_similarity(refs, preds)

        noisy_preds = []
        for s in data:
            with tempfile.NamedTemporaryFile(suffix=".wav") as tmp:
                sf.write(tmp.name, add_noise(s["audio"]), 16000)
                out = model.transcribe([tmp.name], source_lang="en", target_lang="en")
                text = out[0] if isinstance(out[0], str) else out[0].text
                noisy_preds.append(text.strip().upper())

        robust = jiwer.wer(refs, noisy_preds)

        del model
        gc.collect()
        torch.cuda.empty_cache()

        return {
            "model": "Canary 1B Flash (NeMo)",
            "wer": wer * 100,
            "semantic_similarity": sem,
            "avg_rtf": float(np.mean(rtfs)),
            "robust_wer": robust * 100,
            "load_time": load_time,
            "avg_inference_time": float(np.mean(times)),
            "total_inference_time": float(sum(times)),
            "model_size": "~1.2GB",
            "notes": "Official NVIDIA NeMo loader"
        }


    # --------------------------
    # RUN ALL
    # --------------------------
    def run(self, data):
        results = []

        results.append(self.eval_transformers("openai/whisper-base", "Whisper Base", data, "~140MB", "Fastest baseline"))
        results.append(self.eval_transformers("distil-whisper/distil-large-v3", "Distil-Whisper Large v3", data, "~756MB", "Best tradeoff"))
        results.append(self.eval_transformers("openai/whisper-large-v3", "Whisper Large v3", data, "~1.5GB", "Highest accuracy"))
        results.append(self.eval_transformers("facebook/wav2vec2-large-960h-lv60-self", "wav2vec2 Large", data, "~1.2GB", "SSL model"))
        results.append(self.eval_canary_nemo(data))

        return pd.DataFrame(results)


# ==========================
# MAIN
# ==========================
if __name__ == "__main__":

    benchmark = STTBenchmark()

    test_data = benchmark.load_librispeech_dataset(
        base_path="/kaggle/input/benchmark/LibriSpeech/test-clean",
        max_samples=1000
    )

    df = benchmark.run(test_data)

    print("\n================ RESULTS ================")
    print(df.to_string(index=False))

    df.to_csv("stt_benchmark_results_extended.csv", index=False)
    print("\nSaved → stt_benchmark_results_extended.csv")


Loaded 1000 random samples

Evaluating: Whisper Base
Saved predictions → predictions_whisper_base.csv

Evaluating: Distil-Whisper Large v3
Saved predictions → predictions_distil-whisper_large_v3.csv

Evaluating: Whisper Large v3
Saved predictions → predictions_whisper_large_v3.csv

Evaluating: wav2vec2 Large
Saved predictions → predictions_wav2vec2_large.csv

Evaluating: Canary 1B Flash (NeMo)
[NeMo I 2026-01-25 15:52:01 nemo_logging:393] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2026-01-25 15:52:01 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1152 tokens
[NeMo I 2026-01-25 15:52:01 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-25 15:52:01 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-25 15:52:01 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2026-01-25 15:52:01 nemo_logging:393] Tokenizer SentencePieceTokeni

[NeMo W 2026-01-25 15:52:02 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 8
    pin_memory: true
    prompt_format: canary2
    max_tps: 25
    max_duration: 40.0
    text_field: answer
    lang_field: target_lang
    use_bucketing: true
    bucket_duration_bins:
    - - 3.971
      - 30
    - - 3.971
      - 48
    - - 4.973
      - 37
    - - 4.973
      - 60
    - - 5.85
      - 42
    - - 5.85
      - 71
    - - 6.56
      - 46
    - - 6.56
      - 79
    - - 7.32
      - 49
    - - 7.32
      - 88
    - - 8.19
      - 54
    - - 8.19
      - 99
    - - 8.88
      - 61
    - - 8.88
      - 107
    - - 9.76
      - 66
    - - 9.76
      - 118
    - - 10.56
      - 72


[NeMo I 2026-01-25 15:52:02 nemo_logging:393] PADDING: 0


ERROR:hydra.utils:Error getting class at nemo.collections.asr.modules.transformer.get_nemo_transformer: Located non-class of type 'function' while loading 'nemo.collections.asr.modules.transformer.get_nemo_transformer'


[NeMo I 2026-01-25 15:52:15 nemo_logging:393] Model EncDecMultiTaskModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--canary-1b-flash/snapshots/a9a55e0295e7dd50d0c8c2a19491900a0daf24f3/canary-1b-flash.nemo.


[NeMo W 2026-01-25 15:52:16 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: enable_chunking,trim_silence
[NeMo W 2026-01-25 15:52:16 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 1it [00:00,  4.39it/s]
[NeMo W 2026-01-25 15:52:16 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: enable_chunking,trim_silence
[NeMo W 2026-01-25 15:52:16 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the 

Saved predictions → predictions_canary_1b_flash.csv


[NeMo W 2026-01-25 15:58:14 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: enable_chunking,trim_silence
[NeMo W 2026-01-25 15:58:14 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 1it [00:00,  5.10it/s]
[NeMo W 2026-01-25 15:58:15 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: enable_chunking,trim_silence
[NeMo W 2026-01-25 15:58:15 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the 


================ RESULTS ================
                  model       wer  semantic_similarity  avg_rtf  robust_wer  load_time  avg_inference_time  total_inference_time model_size                       notes
           Whisper Base 17.607860             0.918159 0.050612   17.773238   0.866333            0.336692            336.691619     ~140MB            Fastest baseline
Distil-Whisper Large v3 11.386741             0.940653 0.109905   10.496619   1.492785            0.613819            613.818621     ~756MB               Best tradeoff
       Whisper Large v3 10.564716             0.940918 0.462857    9.917798   2.868547            2.824130           2824.129506     ~1.5GB            Highest accuracy
         wav2vec2 Large  1.809427             0.977953 0.009773    1.955348   1.312349            0.066454             66.454145     ~1.2GB                   SSL model
 Canary 1B Flash (NeMo) 14.689430             0.959799 0.052756   14.616470  17.585274            0.350369           